In [ ]:
(***hide***)

#r "nuget: Expecto, 9.0.4"
#r "nuget: FSharpAux, 2.0.0"
#r "nuget: FsSpreadsheet, 5.0.2"
#r "nuget: FsSpreadsheet.ExcelIO, 5.0.2"
#r "nuget: ARCTokenization, 3.0.0"
#r "nuget: Graphoscope, 0.6.0-preview.1"
#r "nuget: OBO.NET, 0.4.2"
#r "nuget: Cytoscape.NET, 0.2.0"
#r "nuget: AnyBadge.NET, 2.0.0"
#r @"C:\Users\schne\source\repos\nfdi4plants\arc-validate\src\ARCExpect\bin\Release\netstandard2.0\ARCExpect.dll"


# ARCExpect



## Introduction

`ARCExpect` offers easy to use and understand APIs to create and execute validation cases. The main intention of this library is offering a framework for validation of [ControlledVocabulary](https://github.com/nfdi4plants/ARCTokenization) tokens in **Research Data management (RDM)** based on testing principles from the world of software development:

- a `validation case` is the equivalent of a unit test
- `Validate.*` functions are the equivalent of `Assert.*`, `Expect.*`, or equivalent functions
- A `BadgeCreation` API based on [AnyBadge.NET](https://github.com/kMutagene/AnyBadge.NET) enables creation of badges that visualize the validation results.
- export of the validation results as a `junit` xml file enables further integration into e.g. CI/CD pipelines

**User-facing APIs:**

- `Validate` :
  - validate **ControlledVocabulary** tokens, e.g. for their compliance with a reference ontology or for the type and shape of annotated value
- `BadgeCreation`:
  - Create and style small .svg images that visualize the validation results of a validation suite
- `Execute` - perform actions (e.g. running validation, writing files result to disk)

**Additional APIs:**

- `OBOGraph`
  - Create, complete, and query the graph representation of an ontology
- `ARCGraph`
  - Create, complete, and query a graph of ControlledVocabulary tokens based on the relations in a reference ontology

## Creating a validation case

ARCExpect offers a `validationCase` Computation expression that enables the creation of validation cases in a very concise and readable way.

Suppose you have a metadata token that annotates the value 42 with the term "Thing" from the "Some Ontology" ontology (meaning this token represents "42 Things"). 

In [8]:
open ControlledVocabulary

let testParam = 
    CvParam(
        CvTerm.create("SO:000001","Thing","Some Ontology"),
        ParamValue.Value 42
    )

Use the `validationCase` CE to creater a validation case, and use the `Validate` API to specify the validation step:

In [11]:
let myValidationCase = 
    validationCase (TestID.Name "has value 42") {
        testParam |> Validate.Param.ValueIsEqualTo 42
    }


this will create a validation case that checks if the value of the given token is 42.

Let's take a closer look at some API specifics: The `Validate` API is designed to be very close to natural language. 

`Validate.Param.TermIsEqualTo 42` means:

> "validate (for a) Param (that its) value is equal to 42".

Where `Param` is any type that implements the `IParam` interface from `ControlledVocabulary` - an interface that can be used to represent a value annotated with some controlled vocabulary.

## Performing validation

Use the `Execute` API to perform the validation, obtaining a summary object

In [19]:
let validationResults = Execute.Validation(myValidationCase)

validationResults.results
|> List.head
|> snd

{ result = Passed\n count = 1\n meanDuration = 0.0\n maxDuration = 0.0 } result Passed tag Passed order 1 isPassed Passed isIgnored Passed isFailed Passed isException False count 1 meanDuration 0 maxDuration 0 duration 00:00:00

## Creating validation badges

Use the `BadgeCreation` API to create badges that visualize the validation results.

In [25]:
let myBadge = 
    validationResults 
    |> BadgeCreation.ofTestResults(
        labelText = "My validation"
    )

(***hide***)
myBadge.BadgeSvgText
|> DisplayFunctions.HTML

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 My validation 
 My validation 
 
 
 1/1 
 1/1

You can also use `Execute.BadgeCreation` to write directly to a file:

In [ ]:
validationResults 
|> Execute.BadgeCreation(
    path = "some/path/here.svg",
    labelText = "My validation"
)

## Exporting JUnit XML

Use the `Execute.JUnitSummaryCreation` API to export the validation results as a `junit` xml file.


In [ ]:
validationResults
|> Execute.JUnitSummaryCreation(
    path = "some/path/here.xml"
)

## Performing a ValidationPipeline

use the `Execute.ValidationPipeline` API to perform a validation pipeline consisting of these steps:

- run the validation cases and collect the results
- create the badge at the given path
- export the junit summary at the given path

In [28]:
myValidationCase
|> Execute.ValidationPipeline(
    jUnitPath = "some/path/here.xml",
    badgePath = "some/path/here.svg",
    labelText = "My validation"
)